In [623]:
# Importing necessary libraries

import numpy as np
import pandas as pd
import os
import cmath

In [624]:
# Reading bus data into panda dataframes

def r_buses(file):
    df_buses = pd.read_csv(file, sep=";")
    V = df_buses.V.to_numpy()
    delta = df_buses.Angle.to_numpy()
    P = df_buses.P_gen.to_numpy()
    Q = df_buses.Q_gen.to_numpy()
    bus_type = df_buses.BusType.to_numpy()
    return V, delta, P, Q, bus_type


V, delta, P, Q, bus_type = r_buses('Busdata.csv')

print(V)
print(delta)
print(P)
print(Q)
print(bus_type)


# Creating additional vectors that will be used and updaed during the calculations
V_calc, delta_calc, P_calc, Q_calc, bus_type = r_buses('Busdata.csv')

for x in range(len(V_calc)):
    if (np.isnan(V_calc[x])):
        V_calc[x] = 1
    if(np.isnan(delta_calc[x])):
        delta_calc[x] = 0



[ 1. nan  1. nan nan]
[nan nan  0. nan nan]
[ 1.  -0.6  nan -0.6 -0.5]
[ nan -0.3  nan -0.2 -0.4]
[1 2 0 2 2]


In [625]:
# Function to create the Y-bus matrix
# shape needs to be specified per now. Fix this?
def Ybus(file, shape):
    df_impedances = pd.read_csv(file, sep=";")
    print(df_impedances.head())
    Z_values = np.zeros((shape,shape), dtype=complex)
    Y_bus = np.zeros((shape,shape), dtype=complex)

    for x in range(df_impedances.shape[0]):
        from_line = df_impedances['From_line'][x]
        to_line = df_impedances['To_line'][x]

        # Adding the diagonal elements to the Y-bus
        Y_bus[from_line-1][from_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[from_line-1][from_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/(df_impedances['R'][x] + df_impedances['X'][x]*1j)
        Y_bus[to_line-1][to_line-1] += 1/2*(df_impedances['Full_line_B'][x]*1j)

        # Z values for off diagonal elements
        Z_values[from_line-1][to_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
        Z_values[to_line-1][from_line-1] = df_impedances['R'][x] + df_impedances['X'][x]*1j
    # Adding off diagonal elements
    for i in range(shape):
        for j in range(shape):
            if(Z_values[i][j] != 0. +0.j):
                if(i != j):
                    Y_bus[i][j] = - 1/Z_values[i][j]
            else:
                if(i != j):
                    Y_bus[i][j] = Z_values[i][j]
    return Y_bus

In [626]:
Ybus = Ybus('impedances.csv', 5)
print(Ybus)

   From_line  To_line      R      X  Full_line_B
0          1        2  0.050  0.250       0.0500
1          2        3  0.025  0.125       0.1000
2          1        4  0.020  0.200       0.0333
3          2        4  0.020  0.200       0.0333
4          4        5  0.010  0.100       0.0200
[[ 1.26428027 -8.7549989j  -0.76923077 +3.84615385j
   0.         +0.j         -0.4950495  +4.95049505j
   0.         +0.j        ]
 [-0.76923077 +3.84615385j  2.80274181-16.39730659j
  -1.53846154 +7.69230769j -0.4950495  +4.95049505j
   0.         +0.j        ]
 [ 0.         +0.j         -1.53846154 +7.69230769j
   1.53846154 -7.64230769j  0.         +0.j
   0.         +0.j        ]
 [-0.4950495  +4.95049505j -0.4950495  +4.95049505j
   0.         +0.j          1.98019802-19.7586802j
  -0.99009901 +9.9009901j ]
 [ 0.         +0.j          0.         +0.j
   0.         +0.j         -0.99009901 +9.9009901j
   0.99009901 -9.8909901j ]]


In [627]:
# Litt testkode i denne blokken...

class Network:
    def __init__(self, buses):
        self.buses = buses
    
    def get_V_vec(self):
        V_vec = []
        for x in range(len(self.buses)):
            V_vec.append(self.buses[x].V)
        return V_vec

    def get_P_vec(self):
        P_vec = []
        for x in range(len(self.buses)):
            P_vec.append(self.buses[x].P)
        return P_vec

    def get_Q_vec(self):
        Q_vec = []
        for x in range(len(self.buses)):
            Q_vec.append(self.buses[x].Q)
        return Q_vec

    def get_delta_vec(self):
        delta_vec = []
        for x in range(len(self.buses)):
            delta_vec.append(self.buses[x].delta)
        return delta_vec

    def get_bus_type_vec(self):
        bus_type_vec = []
        for x in range(len(self.buses)):
            bus_type_vec.append(self.buses[x].bus_type)
        return bus_type_vec

    def get_bus_num_vec(self):
        bus_num_vec = []
        for x in range(len(self.buses)):
            bus_num_vec.append(self.buses[x].bus_num)
        return bus_num_vec
    
    def get_V_calc(self):
        V_vec = []
        for x in range(len(self.buses)):
            if(np.isnan(self.buses[x].V)):
                V_vec.append(1)
            else:
                V_vec.append(self.buses[x].V)
        return V_vec

    def get_delta_calc(self):
        delta_vec = []
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].delta)):
                delta_vec.append(0)
            else:
                delta_vec.append(self.buses[x].delta)
        return delta_vec
    
    
    def get_PQ_vec(self):
        PQ_vec = []
        PQ_jacobian = []
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].P) == False):
                PQ_vec.append(self.buses[x].P)
                businfo = PQ("P", self.buses[x].bus_num)
                PQ_jacobian.append(businfo)
        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].Q) == False):
                PQ_vec.append(self.buses[x].Q)
                businfo = PQ("Q", self.buses[x].bus_num)
                PQ_jacobian.append(businfo)
        return PQ_vec, PQ_jacobian

    def get_VD_jacobian(self):
        VD_jacobian = []

        for x in range(len(self.buses)):
            if (np.isnan(self.buses[x].delta)):
                businfo = VD("D", self.buses[x].bus_num)
                VD_jacobian.append(businfo)  

        for x in range(len(self.buses)):    
            if (np.isnan(self.buses[x].V)):
                businfo = VD("V", self.buses[x].bus_num)
                VD_jacobian.append(businfo)

        return VD_jacobian

class Buses:
    def __init__(self, P, Q, V, delta, bus_num, bus_type):
        self.P = P
        self.Q = Q
        self.V = V
        self.delta = delta
        self.bus_num = bus_num 
        self.bus_type = bus_type



In [628]:
def read_buses(file):
    bus_vec = []
    df_buses = pd.read_csv(file, sep=";")
    for x in range(df_buses.shape[0]):
        V = df_buses['V'][x]
        delta = df_buses['Angle'][x]
        P = df_buses['P_gen'][x]
        Q = df_buses['Q_gen'][x]
        bus_type = df_buses['BusType'][x]
        bus_num = df_buses['BusNum'][x]
        new_bus = Buses(P, Q, V, delta, bus_num, bus_type)
        bus_vec.append(new_bus)
        
    return bus_vec

bus_vec = read_buses('Busdata.csv')

power_network = Network(bus_vec)

V_init = power_network.get_V_calc()
delta_init = power_network.get_delta_calc()
P_init = power_network.get_P_vec()
Q_init = power_network.get_Q_vec()
bus_num_init = power_network.get_bus_num_vec()

#Function to caluculate the known P values
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        if (np.isnan(P[i])):
            P_Calc[i] = np.nan
        else:
            for j in range (len(BusNum)):
                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc


#Function to caluculate the known Q values
def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(len(BusNum), dtype=complex)
    for i in range (len(BusNum)):
        if (np.isnan(Q[i])):
            Q_Calc[i] = np.nan
        else:
           for j in range (len(BusNum)):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc




P_calc = P_Calc(V_init, Ybus, bus_num_init, delta_init, P_init)
Q_calc = Q_Calc(V_init, Ybus, bus_num_init, delta_init, Q_init)

print(P_calc)
print(Q_calc)

[5.55111512e-17+0.j 5.55111512e-17+0.j            nan+0.j
 0.00000000e+00+0.j 0.00000000e+00+0.j]
[     nan+0.j -0.09165+0.j      nan+0.j -0.0433 +0.j -0.01   +0.j]


In [629]:
"""

#Function to caluculate the known P values
def P_Calc(V, YBus, BusNum, delta, P):
    P_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if (np.isnan(P[i])):
            P_Calc[i] = np.nan
        else:
            for j in range (BusNum):
                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])
                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) 
    return P_Calc


#Function to caluculate the known Q values
def Q_Calc(V, YBus, BusNum, delta, Q):
    Q_Calc = np.zeros(BusNum, dtype=float)
    for i in range (BusNum):
        if (np.isnan(Q[i])):
            Q_Calc[i] = np.nan
        else:
           for j in range (BusNum):
                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])
                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) 
    return Q_Calc




P_calc = (P_Calc(V_calc, Ybus, num_buses, delta_calc, P))
Q_calc = (Q_Calc(V_calc, Ybus, num_buses, delta_calc, Q))

print(P_calc)
print(Q_calc)

"""

'\n\n#Function to caluculate the known P values\ndef P_Calc(V, YBus, BusNum, delta, P):\n    P_Calc = np.zeros(BusNum, dtype=float)\n    for i in range (BusNum):\n        if (np.isnan(P[i])):\n            P_Calc[i] = np.nan\n        else:\n            for j in range (BusNum):\n                P_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.cos(delta[i]-delta[j])\n                            +np.imag(YBus)[i][j]*cmath.sin(delta[i]-delta[j]))) \n    return P_Calc\n\n\n#Function to caluculate the known Q values\ndef Q_Calc(V, YBus, BusNum, delta, Q):\n    Q_Calc = np.zeros(BusNum, dtype=float)\n    for i in range (BusNum):\n        if (np.isnan(Q[i])):\n            Q_Calc[i] = np.nan\n        else:\n           for j in range (BusNum):\n                Q_Calc[i] += (V[i]*V[j]*(np.real(YBus)[i][j]*cmath.sin(delta[i]-delta[j])\n                                         -np.imag(YBus)[i][j]*cmath.cos(delta[i]-delta[j]))) \n    return Q_Calc\n\n\n\n\nP_calc = (P_Calc(V_calc, Ybus, num_buses,

In [630]:
#Jacobian calculations

class PQ:
    def __init__(self, Bus_type, Bus_num):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num

class VD:
    def __init__(self, Bus_type, Bus_num):
        self.Bus_type = Bus_type
        self.Bus_num = Bus_num

""" 
def jacobian_init(P,Q, V, delta):

    len_vec = len(P) + len(Q)
    len_vec -= np.isnan(P).sum()
    len_vec -= np.isnan(Q).sum()
    print(len_vec)

    PQ_vec =  np.zeros(len_vec, dtype=float)
    PQ_jacobian = []
    VD_jacobian = []
    numP = 0
    numQ = 0
    rowP = 0
    rowQ = 0
    for x in range(len(P)):
        if (np.isnan(P[x]) == False):
            PQ_vec[numP] = P[x]
            businfo = PQ("P", rowP, numP)
            PQ_jacobian.append(businfo)
            numP += 1
        rowP += 1
    for x in range(len(Q)):
        if (np.isnan(Q[x]) == False):
            PQ_vec[numP + numQ] = Q[x]
            businfo = PQ("Q", rowQ, numQ+numP)
            PQ_jacobian.append(businfo)
            numQ += 1
        rowQ += 1

    numV = 0
    numT = 0
    rowV = 0
    rowT = 0
    for x in range(len(delta)):
        if (np.isnan(delta[x])):
            businfo = VD("T", rowT, numT)
            VD_jacobian.append(businfo)
            numT += 1
        rowT += 1    

    for x in range(len(V)):    
        if (np.isnan(V[x])):
            businfo = VD("V", rowV, numV)
            VD_jacobian.append(businfo)
            numV += 1
        rowV += 1

    return VD_jacobian, PQ_jacobian, PQ_vec
"""


PQ_vec, PQ_jacobian = power_network.get_PQ_vec()

VD_jacobian = power_network.get_VD_jacobian()




def make_jacobian(VD_jacobian, PQ_jacobian, PQ_vec, num_buses, V, delta):
    j = np.zeros((7,7), dtype=complex)

    #V = [1,1,1,1,1]
    #delta = [0,0,0,0,0]
    
    for x in range(len(PQ_vec)):
        for y in range(len(PQ_vec)):
            if (PQ_jacobian[x].Bus_num == VD_jacobian[y].Bus_num):
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'D'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 0
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(-np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))      
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'V'):
                    for i in range(num_buses):
                        if (i==PQ_jacobian[x].Bus_num):
                            j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,i])
                        else:
                            j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i])+np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i]))      

                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[y]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))+np.imag(Ybus[PQ_jacobian[x].Bus_num,i]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i]))*1j)

                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += 2*V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,i])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[i])-np.imag(Ybus[PQ_jacobian[x].Bus_num,i])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[i]))
           
            else:
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VD_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*1j*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'P' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])+np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num])*1j*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'V'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num])-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))*1j)
                
                if (PQ_jacobian[x].Bus_type == 'Q' and VD_jacobian[y].Bus_type == 'D'):
                    j[x,y] += V[PQ_jacobian[x].Bus_num]*V[VD_jacobian[y].Bus_num]*(np.real(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.cos(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))-np.imag(Ybus[PQ_jacobian[x].Bus_num,VD_jacobian[y].Bus_num]*cmath.sin(delta[PQ_jacobian[x].Bus_num]-delta[VD_jacobian[y].Bus_num]))*1j)
    return j



j = make_jacobian(VD_jacobian, PQ_jacobian, PQ_vec, num_buses, V_calc, delta_calc)

print(j)


    

[[ 0.         +8.7966489j   0.         -3.84615385j
   0.         -4.95049505j  0.         +0.j
  -0.76923077 +0.j         -0.4950495  +0.j
   0.         +0.j        ]
 [ 0.         -3.84615385j  0.        +16.48895659j
   0.         -4.95049505j  0.         +0.j
   2.80274181 +0.j         -0.4950495  +0.j
   0.         +0.j        ]
 [ 0.         -4.95049505j  0.         -4.95049505j
   0.        +19.8019802j   0.         -9.9009901j
  -0.4950495  +0.j          1.98019802 +0.j
  -0.99009901 +0.j        ]
 [ 0.         +0.j          0.         +0.j
   0.         -9.9009901j   0.         +9.9009901j
   0.         +0.j         -0.99009901 +0.j
   0.99009901 +0.j        ]
 [-0.76923077 +0.j          0.         +0.j
  -0.4950495  +0.j          0.         +0.j
   0.         +0.j          0.         +0.j
   0.         +0.j        ]
 [-0.4950495  +0.j         -0.4950495  +0.j
  -0.99009901 +0.j         -0.99009901 +0.j
   0.         +0.j          0.        -19.8019802j
   0.         +0.j     

In [631]:
# Inverse of jacobian
j_inv = np.linalg.inv(j)
print(j_inv)

[[ 0.00000000e+00-8.99503586e-02j  0.00000000e+00-2.99180829e-02j
   0.00000000e+00-2.96133582e-02j  0.00000000e+00-2.97649618e-02j
  -6.12511743e-01-0.00000000e+00j  1.52362330e-03+0.00000000e+00j
   7.58784103e-06+0.00000000e+00j]
 [ 0.00000000e+00+8.82561245e-01j  0.00000000e+00+3.81187638e-01j
   0.00000000e+00+7.86744078e-01j  0.00000000e+00+7.85969734e-01j
  -3.12852152e+00-0.00000000e+00j  7.78219902e-03+0.00000000e+00j
   3.87563572e-05+0.00000000e+00j]
 [ 0.00000000e+00+1.39769019e-01j  0.00000000e+00+4.64880980e-02j
   0.00000000e+00+4.60146028e-02j  0.00000000e+00+4.62501714e-02j
  -1.06825098e+00-0.00000000e+00j -2.36747621e-03-0.00000000e+00j
  -1.17903376e-05-0.00000000e+00j]
 [ 0.00000000e+00+1.43148219e-01j  0.00000000e+00+4.78311461e-02j
   0.00000000e+00+4.83675637e-02j  0.00000000e+00-5.23963163e-02j
  -1.08828598e+00-0.00000000e+00j  2.68208806e-03+0.00000000e+00j
   5.04328772e-03+0.00000000e+00j]
 [ 5.06280623e+00+0.00000000e+00j  2.55593819e+00+0.00000000e+00j
  

In [632]:
# Computing the differences
VT_jacobian_calc, PQ_jacobian_calc, PQ_vec_calc = jacobian_init(P_calc, Q_calc, V_calc, delta_calc)

PQ_diff = PQ_vec - PQ_vec_calc

print(PQ_diff)

7


C:\Users\asbjo\AppData\Local\Temp\ipykernel_11348\3791294645.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
  PQ_vec[numP] = P[x]


TypeError: __init__() takes 3 positional arguments but 4 were given

In [488]:

delta_x = np.matmul(j_inv,PQ_diff)

print(delta_x)

[ 0.12737511-0.03934901j  0.65059287-0.21118265j  0.22294567+0.06114231j
  0.22435722+0.11162715j -1.51091326-3.25296437j -0.00237518+0.03390099j
 -0.00252679+0.01978556j]


In [502]:
initial_guess = [0,0,0,0,1,1,1]

new_x = initial_guess + delta_x

print(new_x)

[ 0.12737511-0.03934901j  0.65059287-0.21118265j  0.22294567+0.06114231j
  0.22435722+0.11162715j -0.51091326-3.25296437j  0.99762482+0.03390099j
  0.99747321+0.01978556j]


In [503]:
num_var = 0
new_delta = []
new_V = []

for x in range(len(delta)):
    if (np.isnan(delta[x])):
        new_delta[x] = new_x[num_var]
        num_var += 1
    else: 
        new_delta[x] = delta[x]

for x in range(len(V)):
    if (np.isnan(v[x])):
        new_V[x] = new_x[num_var]
        num_var += 1
    else:
        new_V[x] = V[4+x]

print(new_V, new_delta)


IndexError: list assignment index out of range